In [ ]:
pip install pyspark


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 4.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.2-py2.py3-none-any.whl size=317812365 sha256=23c1df278a85b3cb8812e36ec0919162355e31769636163e9d44441f7a3adf4b
  Stored in directory: /root/.cache/pip/wheels/34/34/bd/03944534c44b677cd5859f248090daa9fb27b3c8f8e5f49574
Successfully built pyspark


# Building a KMeans Clustering algorithm on a Spark cluster.

In [ ]:
from pyspark.sql import SparkSession

In [ ]:
# Create a Spark session
spark = SparkSession.builder.appName("HousingData").getOrCreate()


# Read the CSV file into a DataFrame
df = spark.read.option("header", "true").option("inferSchema", "true").csv('/content/housing.csv')

# Inspect the schema of the DataFrame
df.printSchema()


root
 |-- CRIM: double (nullable = true)
 |-- ZN: double (nullable = true)
 |-- INDUS: double (nullable = true)
 |-- CHAS: integer (nullable = true)
 |-- NOX: double (nullable = true)
 |-- RM: double (nullable = true)
 |-- AGE: double (nullable = true)
 |-- DIS: double (nullable = true)
 |-- RAD: integer (nullable = true)
 |-- TAX: double (nullable = true)
 |-- PTRATIO: double (nullable = true)
 |-- B: double (nullable = true)
 |-- LSTAT: double (nullable = true)
 |-- MEDV: double (nullable = true)



In [ ]:
from pyspark.sql.functions import col

df = df.withColumn("TAX", col("TAX").cast("integer"))

# Inspect the schema to confirm the change
df.printSchema()

root
 |-- CRIM: double (nullable = true)
 |-- ZN: double (nullable = true)
 |-- INDUS: double (nullable = true)
 |-- CHAS: integer (nullable = true)
 |-- NOX: double (nullable = true)
 |-- RM: double (nullable = true)
 |-- AGE: double (nullable = true)
 |-- DIS: double (nullable = true)
 |-- RAD: integer (nullable = true)
 |-- TAX: integer (nullable = true)
 |-- PTRATIO: double (nullable = true)
 |-- B: double (nullable = true)
 |-- LSTAT: double (nullable = true)
 |-- MEDV: double (nullable = true)



In [ ]:
# Unfortunately, Spark did not infer the schema correctly. We will have to manually create a schema.
# I will Create the schema using SructType() and StructField().

from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType

In [ ]:

#### REMOVE THIS ONE MAYBE, & the CELL ABOVE AS WELL

##########schema = StructType([
    StructField("CRIM", DoubleType(), True),
    StructField("ZN", DoubleType(), True),
    StructField("INDUS", DoubleType(), True),
    StructField("CHAS", IntegerType(), True),
    StructField("NOX", DoubleType(), True),
    StructField("RM", DoubleType(), True),
    StructField("AGE", DoubleType(), True),
    StructField("DIS", DoubleType(), True),
    StructField("RAD", IntegerType(), True),
    StructField("TAX", IntegerType(), True),
    StructField("PTRATIO", DoubleType(), True),
    StructField("B", DoubleType(), True),
    StructField("LSTAT", DoubleType(), True),
    StructField("MEDV", DoubleType(), True)
])


In [ ]:
df.show(20 , truncate = False)

+-------+----+-----+----+-----+-----+-----+------+---+---+-------+------+-----+----+
|CRIM   |ZN  |INDUS|CHAS|NOX  |RM   |AGE  |DIS   |RAD|TAX|PTRATIO|B     |LSTAT|MEDV|
+-------+----+-----+----+-----+-----+-----+------+---+---+-------+------+-----+----+
|0.00632|18.0|2.31 |0   |0.538|6.575|65.2 |4.09  |1  |296|15.3   |396.9 |4.98 |24.0|
|0.02731|0.0 |7.07 |0   |0.469|6.421|78.9 |4.9671|2  |242|17.8   |396.9 |9.14 |21.6|
|0.02729|0.0 |7.07 |0   |0.469|7.185|61.1 |4.9671|2  |242|17.8   |392.83|4.03 |34.7|
|0.03237|0.0 |2.18 |0   |0.458|6.998|45.8 |6.0622|3  |222|18.7   |394.63|2.94 |33.4|
|0.06905|0.0 |2.18 |0   |0.458|7.147|54.2 |6.0622|3  |222|18.7   |396.9 |5.33 |36.2|
|0.02985|0.0 |2.18 |0   |0.458|6.43 |58.7 |6.0622|3  |222|18.7   |394.12|5.21 |28.7|
|0.08829|12.5|7.87 |0   |0.524|6.012|66.6 |5.5605|5  |311|15.2   |395.6 |12.43|22.9|
|0.14455|12.5|7.87 |0   |0.524|6.172|96.1 |5.9505|5  |311|15.2   |396.9 |19.15|27.1|
|0.21124|12.5|7.87 |0   |0.524|5.631|100.0|6.0821|5  |311|15.2   

In [ ]:
# Removing any missing values from the data

df.na.drop()

DataFrame[CRIM: double, ZN: double, INDUS: double, CHAS: int, NOX: double, RM: double, AGE: double, DIS: double, RAD: int, TAX: int, PTRATIO: double, B: double, LSTAT: double, MEDV: double]

In [ ]:
 # Identifying the column to define the label column which is 'MEDV'

 df.columns

['CRIM',
 'ZN',
 'INDUS',
 'CHAS',
 'NOX',
 'RM',
 'AGE',
 'DIS',
 'RAD',
 'TAX',
 'PTRATIO',
 'B',
 'LSTAT',
 'MEDV']

In [ ]:
# Assembling the feature columns as a Vector Column.
# Spark MLlib requires input features to be assembled into a single vector column named "features"
from pyspark.ml.feature import VectorAssembler
assembler = VectorAssembler(inputCols=['CRIM',
 'ZN',
 'INDUS',
 'CHAS',
 'NOX',
 'RM',
 'AGE',
 'DIS',
 'RAD',
 'TAX',
 'PTRATIO',
 'B',
 'LSTAT'], outputCol='features')

In [52]:
# Transform the DataFrame
transformed_df = assembler.transform(df)

# Print out the newly transformed DataFrame
transformed_df.show()


# the features column at the end will be used to fit the Linear Regression model while our label will be 'MEDV'

+-------+----+-----+----+-----+-----+-----+------+---+---+-------+------+-----+----+--------------------+
|   CRIM|  ZN|INDUS|CHAS|  NOX|   RM|  AGE|   DIS|RAD|TAX|PTRATIO|     B|LSTAT|MEDV|            features|
+-------+----+-----+----+-----+-----+-----+------+---+---+-------+------+-----+----+--------------------+
|0.00632|18.0| 2.31|   0|0.538|6.575| 65.2|  4.09|  1|296|   15.3| 396.9| 4.98|24.0|[0.00632,18.0,2.3...|
|0.02731| 0.0| 7.07|   0|0.469|6.421| 78.9|4.9671|  2|242|   17.8| 396.9| 9.14|21.6|[0.02731,0.0,7.07...|
|0.02729| 0.0| 7.07|   0|0.469|7.185| 61.1|4.9671|  2|242|   17.8|392.83| 4.03|34.7|[0.02729,0.0,7.07...|
|0.03237| 0.0| 2.18|   0|0.458|6.998| 45.8|6.0622|  3|222|   18.7|394.63| 2.94|33.4|[0.03237,0.0,2.18...|
|0.06905| 0.0| 2.18|   0|0.458|7.147| 54.2|6.0622|  3|222|   18.7| 396.9| 5.33|36.2|[0.06905,0.0,2.18...|
|0.02985| 0.0| 2.18|   0|0.458| 6.43| 58.7|6.0622|  3|222|   18.7|394.12| 5.21|28.7|[0.02985,0.0,2.18...|
|0.08829|12.5| 7.87|   0|0.524|6.012| 66.6|5.5

**Split the data into the training and test set**

In [61]:
train, test = transformed_df.randomSplit([0.7, 0.3])

In [62]:
train.show(5 , False)

train.count()   # number of records in the training set

+-------+----+-----+----+-----+-----+----+------+---+---+-------+------+-----+----+--------------------------------------------------------------------------+
|CRIM   |ZN  |INDUS|CHAS|NOX  |RM   |AGE |DIS   |RAD|TAX|PTRATIO|B     |LSTAT|MEDV|features                                                                  |
+-------+----+-----+----+-----+-----+----+------+---+---+-------+------+-----+----+--------------------------------------------------------------------------+
|0.00632|18.0|2.31 |0   |0.538|6.575|65.2|4.09  |1  |296|15.3   |396.9 |4.98 |24.0|[0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.09,1.0,296.0,15.3,396.9,4.98]   |
|0.00906|90.0|2.97 |0   |0.4  |7.088|20.8|7.3073|1  |285|15.3   |394.72|7.85 |32.2|[0.00906,90.0,2.97,0.0,0.4,7.088,20.8,7.3073,1.0,285.0,15.3,394.72,7.85]  |
|0.01311|90.0|1.22 |0   |0.403|7.249|21.9|8.6966|5  |226|17.9   |395.93|4.81 |35.4|[0.01311,90.0,1.22,0.0,0.403,7.249,21.9,8.6966,5.0,226.0,17.9,395.93,4.81]|
|0.0136 |75.0|4.0  |0   |0.41 |5.888|47.6|7.31

362

In [63]:
test.show(5 , False)

test.count()

+-------+----+-----+----+-----+-----+----+------+---+---+-------+------+-----+----+--------------------------------------------------------------------------+
|CRIM   |ZN  |INDUS|CHAS|NOX  |RM   |AGE |DIS   |RAD|TAX|PTRATIO|B     |LSTAT|MEDV|features                                                                  |
+-------+----+-----+----+-----+-----+----+------+---+---+-------+------+-----+----+--------------------------------------------------------------------------+
|0.01096|55.0|2.25 |0   |0.389|6.453|31.9|7.3073|1  |300|15.3   |394.72|8.23 |22.0|[0.01096,55.0,2.25,0.0,0.389,6.453,31.9,7.3073,1.0,300.0,15.3,394.72,8.23]|
|0.01301|35.0|1.52 |0   |0.442|7.241|49.3|7.0379|1  |284|15.5   |394.74|5.49 |32.7|[0.01301,35.0,1.52,0.0,0.442,7.241,49.3,7.0379,1.0,284.0,15.5,394.74,5.49]|
|0.0187 |85.0|4.15 |0   |0.429|6.516|27.7|8.5353|4  |351|17.9   |392.43|6.36 |23.1|[0.0187,85.0,4.15,0.0,0.429,6.516,27.7,8.5353,4.0,351.0,17.9,392.43,6.36] |
|0.01965|80.0|1.76 |0   |0.385|6.23 |31.5|9.08

144

**Creating the linear regression model**

In [58]:
from pyspark.ml.regression import LinearRegression

In [64]:
# Create a Linear Regression Estimator
lr = LinearRegression(featuresCol="features", labelCol="MEDV", predictionCol="prediction")

lr_model = lr.fit(train)





*   Test the linear regression model.




In [66]:
# Use the transform method to create predictions
predictions = lr_model.transform(test)

# Print out the newly transformed DataFrame with predictions
predictions.show()


+-------+----+-----+----+------+-----+----+------+---+---+-------+------+-----+----+--------------------+------------------+
|   CRIM|  ZN|INDUS|CHAS|   NOX|   RM| AGE|   DIS|RAD|TAX|PTRATIO|     B|LSTAT|MEDV|            features|        prediction|
+-------+----+-----+----+------+-----+----+------+---+---+-------+------+-----+----+--------------------+------------------+
|0.01096|55.0| 2.25|   0| 0.389|6.453|31.9|7.3073|  1|300|   15.3|394.72| 8.23|22.0|[0.01096,55.0,2.2...|28.115511391065304|
|0.01301|35.0| 1.52|   0| 0.442|7.241|49.3|7.0379|  1|284|   15.5|394.74| 5.49|32.7|[0.01301,35.0,1.5...| 30.34563537804119|
| 0.0187|85.0| 4.15|   0| 0.429|6.516|27.7|8.5353|  4|351|   17.9|392.43| 6.36|23.1|[0.0187,85.0,4.15...|  26.0808247257684|
|0.01965|80.0| 1.76|   0| 0.385| 6.23|31.5|9.0892|  1|241|   18.2| 341.6|12.93|20.1|[0.01965,80.0,1.7...|20.677043374983683|
|0.02187|60.0| 2.93|   0| 0.401|  6.8| 9.9|6.2196|  1|265|   15.6|393.37| 5.03|31.1|[0.02187,60.0,2.9...|33.006885599082096|
